# Validate PR #248 to refactor command interrupt handling

This notebook compares the commands generated using PR #248 to those from
current flight (kadi 6.0.1). 

The notebook assumes that a new commands archive
(`cmds2.h5, cmds2.pkl`) has been created in the current directory using
`utils.migrate_cmds_to_cmds2.make_cmds2()`.

All diffs are either expected or reveal issues in the current flight
processing to determine observation intervals (`OBS` `LOAD_EVT` commands).

In [1]:
import os
from pathlib import Path

import numpy as np

from kadi import __version__, logger
from kadi.commands import commands_v2
from kadi.commands.commands_v2 import clear_caches, get_cmds

In [2]:
print(__version__)

6.0.1


In [3]:
logger.setLevel(10)

In [4]:
def get_ok(cmds):
    """PR #248 includes new commands that are not in the old archive so we
    filter them out here."""
    ignore = (cmds["type"] == "LOAD_EVENT") & (
        cmds["event_type"] == "SCHEDULED_STOP_TIME"
    )
    ignore |= (
        (cmds["type"] == "LOAD_EVENT")
        & (cmds["source"] == "CMD_EVT")
        & np.isin(cmds["event_type"], ["LOAD_NOT_RUN", "OBSERVING_NOT_RUN"])
    )
    ignore |= (cmds["tlmsid"] == "CODISASX") & (cmds["source"] == "CMD_EVT")
    return ~ignore

In [5]:
start = "2000:001"
stop = "2022:240"

In [6]:
cmds_flight = get_cmds(start, stop, scenario='flight')
ok = get_ok(cmds_flight)
cmds_flight_ok = cmds_flight[ok]
print(len(commands_v2.PARS_DICT))
print(len(commands_v2.REV_PARS_DICT))

2022-08-26 06:18:05,561 get_cmds: Getting commands from archive only
2022-08-26 06:18:05,743 load_idx_cmds: Loaded /Users/aldcroft/ska/data/kadi/cmds2.h5 with 1416611 commands
2022-08-26 06:18:06,172 load_pars_dict: Loaded /Users/aldcroft/ska/data/kadi/cmds2.pkl with 146263 pars
146263
146263


In [7]:
clear_caches()
del commands_v2.PARS_DICT._val
del commands_v2.REV_PARS_DICT._val

os.environ['KADI'] = os.path.abspath('..')

In [8]:
cmds_local = get_cmds(start, stop, scenario='flight')
ok = get_ok(cmds_local)
cmds_local_ok = cmds_local[ok]
print(len(commands_v2.PARS_DICT))
print(len(commands_v2.REV_PARS_DICT))

2022-08-26 06:18:23,822 get_cmds: Getting commands from archive only
2022-08-26 06:18:23,977 load_idx_cmds: Loaded /Users/aldcroft/git/kadi/cmds2.h5 with 1416689 commands
2022-08-26 06:18:24,304 load_pars_dict: Loaded /Users/aldcroft/git/kadi/cmds2.pkl with 146150 pars
146150
146150


In [9]:
print(len(cmds_flight_ok), len(cmds_local_ok))
print(len(cmds_flight), len(cmds_local))

1416396 1416400
1416507 1416585


In [10]:
# Code to debug problems, leave commented for production
out = "\n".join(cmds_flight_ok.pformat_like_backstop(max_params_width=200))
Path("cmds_flight.txt").write_text(out)
out = "\n".join(cmds_local_ok.pformat_like_backstop(max_params_width=200))
Path("cmds_local.txt").write_text(out)

159237187

### Results of `diff cmds_flight.txt cmds_local.txt`

Change in `obs_stop` for an `OBS` load event. The new (local) version has 
`obs_stop` at the time of the commanded maneuver at 224:02:25, so that is correct for
defining the end of the observation.
```
1413286c1413286
< 2022:223:13:20:43.345 | LOAD_EVENT       | OBS        | AUG0822A | obsid=45339, simpos=-99616, obs_stop=2022:223:14:15:03.094, manvr_start=2022:223:12:40:44.686, targ_att=(-0.12752005, 0.556391476, -0.351473717, 0.742046756), npnt_enab=True, obs_start=2022:223:13:20:
---
> 2022:223:13:20:43.345 | LOAD_EVENT       | OBS        | AUG0822A | obsid=45339, simpos=-99616, obs_stop=2022:224:02:25:00.000, manvr_start=2022:223:12:40:44.686, targ_att=(-0.12752005, 0.556391476, -0.351473717, 0.742046756), npnt_enab=True, obs_start=2022:223:13:20:
```
This `AODSDITH` command did not happen, so `cmds_local` is correct:
```
1413295d1413294
< 2022:223:13:50:40.960 | COMMAND_SW       | AODSDITH   | CMD_EVT  | event=Bright_star_hold, event_date=2022:223:13:49:23, scs=0
```
The new version is correct. The AUG1322A loads have a maneuver starting at 2022:225:11:40:00.
```
1413309c1413308
< 2022:224:02:40:48.033 | LOAD_EVENT       | OBS        | CMD_EVT  | obsid=45339, simpos=-99616, obs_stop=2022:226:19:15:48.867, manvr_start=2022:224:02:25:10.250, targ_att=(-0.24849372, 0.40954561, -0.31440262, 0.81955735), npnt_enab=True, obs_start=2022:224:02:40:48.
---
> 2022:224:02:40:48.033 | LOAD_EVENT       | OBS        | CMD_EVT  | obsid=45339, simpos=-99616, obs_stop=2022:225:11:40:00.000, manvr_start=2022:224:02:25:10.250, targ_att=(-0.24849372, 0.40954561, -0.31440262, 0.81955735), npnt_enab=True, obs_start=2022:224:02:40:48.
```
New version is correct, NSM at 2022:231:18:36:00 stops the observation.
```
1415054c1415053
< 2022:231:17:04:23.363 | LOAD_EVENT       | OBS        | AUG1322A | obsid=45317, simpos=-99616, obs_stop=2022:231:19:36:57.259, manvr_start=2022:231:16:44:24.407, targ_att=(0.649105589, -0.0318838109, -0.225906169, 0.725680205), npnt_enab=True, obs_start=2022:231:17:0
---
> 2022:231:17:04:23.363 | LOAD_EVENT       | OBS        | AUG1322A | obsid=45317, simpos=-99616, obs_stop=2022:231:18:36:00.000, manvr_start=2022:231:16:44:24.407, targ_att=(0.649105589, -0.0318838109, -0.225906169, 0.725680205), npnt_enab=True, obs_start=2022:231:17:0
```
Tiny difference in the target attitude due to timing difference.
```
1415082c1415081
< 2022:231:19:05:05.999 | LOAD_EVENT       | OBS        | CMD_EVT  | obsid=45317, simpos=-99616, obs_stop=2022:232:03:07:00.000, manvr_start=2022:231:18:36:00.000, targ_att=(0.6377368264106795, 0.4497793420951543, 0.2522209483727346, 0.5721668262453579), npnt_enab=Fals
---
> 2022:231:19:05:05.999 | LOAD_EVENT       | OBS        | CMD_EVT  | obsid=45317, simpos=-99616, obs_stop=2022:232:03:07:00.000, manvr_start=2022:231:18:36:00.000, targ_att=(0.6377368264106797, 0.44977934209515424, 0.2522209483727345, 0.572166826245358), npnt_enab=Fals
```
Tiny difference in the target attitude due to timing difference.
```
1415089c1415088
< 2022:232:03:07:00.000 | LOAD_EVENT       | OBS        | CMD_EVT  | obsid=62624, simpos=-99616, obs_stop=2022:232:20:07:21.000, manvr_start=2022:231:18:36:00.000, targ_att=(0.6377368264106795, 0.4497793420951543, 0.2522209483727346, 0.5721668262453579), npnt_enab=Fals
---
> 2022:232:03:07:00.000 | LOAD_EVENT       | OBS        | CMD_EVT  | obsid=62624, simpos=-99616, obs_stop=2022:232:20:07:21.000, manvr_start=2022:231:18:36:00.000, targ_att=(0.6377368264106797, 0.44977934209515424, 0.2522209483727345, 0.572166826245358), npnt_enab=Fals
```
New version is correct, obsid change to 62623 at 234:22:12:12.000. Flight version has
`obs_stop` at `2022:233:17:40:00` but there is really nothing happening then,
so that just seems like a bug.
```
1415105c1415104
< 2022:232:20:32:28.838 | LOAD_EVENT       | OBS        | CMD_EVT  | obsid=62624, simpos=-99616, obs_stop=2022:233:17:40:00.000, manvr_start=2022:232:20:07:31.250, targ_att=(0.70443765, 0.06647616, -0.10211985, 0.69922818), npnt_enab=True, obs_start=2022:232:20:32:28.8
---
> 2022:232:20:32:28.838 | LOAD_EVENT       | OBS        | CMD_EVT  | obsid=62624, simpos=-99616, obs_stop=2022:234:22:12:12.000, manvr_start=2022:232:20:07:31.250, targ_att=(0.70443765, 0.06647616, -0.10211985, 0.69922818), npnt_enab=True, obs_start=2022:232:20:32:28.8
```
New version is correct, flight version was erroneously cutting the ACIS CTI
for the RLTT of command loads that were not run.
```
1415110a1415110,1415114
> 2022:233:18:10:43.000 | ACISPKT          | AA00000000 | CMD_EVT  | event=RTS, event_date=2022:232:03:09:00, scs=135
> 2022:233:18:10:53.000 | ACISPKT          | AA00000000 | CMD_EVT  | event=RTS, event_date=2022:232:03:09:00, scs=135
> 2022:233:18:10:57.000 | ACISPKT          | WSPOW0002A | CMD_EVT  | event=RTS, event_date=2022:232:03:09:00, scs=135
> 2022:233:18:12:00.000 | ACISPKT          | RS_0000001 | CMD_EVT  | event=RTS, event_date=2022:232:03:09:00, scs=135
> 2022:233:18:12:04.000 | COMMAND_SW       | OORMPDS    | CMD_EVT  | event=RTS, event_date=2022:232:03:09:00, msid=OORMPDS, scs=135
```